<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/Load_a_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# https://blog.csdn.net/qq_27590277/article/details/142383849

In [4]:
# !pip install transformers

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


torch.Size([1, 6, 768])


In [9]:
# Assume we have a pre-trained transformer model
pretrained_transformer  # Load a transformer like GPT or BERT
# GPT2Model
model = LanguageModel(pretrained_transformer).cuda()

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Loss function: Binary cross-entropy for pairwise comparisons
def dpo_loss(preferred_score, non_preferred_score):
    return -torch.log(torch.sigmoid(preferred_score - non_preferred_score))

# Example training loop
def train_dpo(model, dataloader, optimizer):
    model.train()

    for batch in dataloader:
        # Assume the batch consists of input pairs and their preference labels
        preferred_inputs = batch['preferred_input_ids'].cuda()  # Preferred response input IDs
        non_preferred_inputs = batch['non_preferred_input_ids'].cuda()  # Non-preferred response input IDs

        preferred_attention_mask = batch['preferred_attention_mask'].cuda()
        non_preferred_attention_mask = batch['non_preferred_attention_mask'].cuda()

        # Forward pass: Compute scores for both preferred and non-preferred responses
        preferred_score = model(preferred_inputs, attention_mask=preferred_attention_mask)
        non_preferred_score = model(non_preferred_inputs, attention_mask=non_preferred_attention_mask)

        # Compute DPO loss
        loss = dpo_loss(preferred_score, non_preferred_score)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Optionally, log loss and other metrics
        print(f"Training loss: {loss.item()}")

# Dataset for DPO (pseudo code)
# You need to prepare a dataset with pairs of responses
dataloader = ...  # Assume we have a dataloader that loads pairs of responses

# Training
for epoch in range(10):
    train_dpo(model, dataloader, optimizer)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [14]:

# Initialize the model with the pre-trained GPT-2
# NO GPU, fall back to cpu
device = torch.device("cpu")
model = LanguageModel(pretrained_transformer).to(device)

# model = LanguageModel(pretrained_transformer).cuda()

# Example of how to use the model
input_text = "How does machine learning work?"
# inputs = tokenizer(input_text, return_tensors='pt').to('cuda')
inputs = tokenizer(input_text, return_tensors='pt').to(device)

# Forward pass to get the score for the input
output_score = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
print(output_score)

tensor([[0.5076]], grad_fn=<AddmmBackward0>)
